In [1]:
import pennylane as qml
from autograd import grad

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
################ Code from the 'Optimization of molecular geometries' tutorial##################
# imports and relevant defines:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

# Simulation starting parameters:
symbols = ["O", "H", "H"]

# this is selected from the tutorial 'Building molecular Hamiltonians', which has the same nulcear-coordinats for water
x = np.array([0.028,  0.054,  0.10,
              0.986,  1.610, -0.10,
              1.855,  0.002,  0.20], requires_grad=True)
# these parameters are used to match up with the parameters for the VQE run on water in the paper: https://arxiv.org/pdf/2106.13840.pdf
active_electrons = 8
active_orbitals = 6

# define the hamiltonian needed to compute cost-function
def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons,
                                           active_orbitals = active_orbitals)[0]

#hf = qml.qchem.hf_state(electrons=active_electrons, orbitals=active_orbitals)
#print(hf)

In [3]:
# Need to run this: functions to allow us to calculate angle between OH1 and OH2:
import math

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return (np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180/math.pi) #convert to degrees

In [4]:
################ Code from the 'Building the adaptive circuit' tutorial below:################
active_electrons = active_electrons
active_orbitals = active_orbitals

H, qubits = qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons, active_orbitals = active_orbitals)
singles, doubles = qchem.excitations(active_electrons, qubits)

hf_state = qchem.hf_state(active_electrons, qubits)
print(hf_state)



#compute the significant double-excitation gates:
def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)
print("Computed gradients for all possible Double Excitation Gates: \n")
for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")   
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected double-excitation gates: ", len(doubles_select))



# optimizing the parameters for the double-excitation gates for Ansatz-wavefunction construction
opt = qml.GradientDescentOptimizer(stepsize=0.5)
params_doubles = np.zeros(len(doubles_select), requires_grad=True)
for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)
print("Done!")



#compute the significant single-excitation gates:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)


cost_fn = qml.QNode(circuit_2, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)
grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)
print("Computed gradients for all possible Single Excitation Gates: \n")
for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected single-excitation gates: ", len(singles_select))

#Total Number of Gates selected to construct the Quantum Ansatz:
print("Total selected gates: "+  str(len(doubles_select) + len(singles_select)))

################# End code from the 'Building the adaptive circuit' tutorial ##################

[1 1 1 1 1 1 1 1 0 0 0 0]
Computed gradients for all possible Double Excitation Gates: 

Excitation : [0, 1, 8, 9], Gradient: -0.11187677542286938
Excitation : [0, 1, 8, 11], Gradient: -0.001387070013415123
Excitation : [0, 1, 9, 10], Gradient: 0.001387070013415123
Excitation : [0, 1, 10, 11], Gradient: -0.056213035035785376
Excitation : [0, 2, 8, 10], Gradient: 0.0001452588867044751
Excitation : [0, 3, 8, 9], Gradient: -0.04473596626384681
Excitation : [0, 3, 8, 11], Gradient: -0.0016024818853710187
Excitation : [0, 3, 9, 10], Gradient: 0.0014572229986665392
Excitation : [0, 3, 10, 11], Gradient: -0.0040843852128207205
Excitation : [0, 4, 8, 10], Gradient: -0.03480327512070891
Excitation : [0, 5, 8, 9], Gradient: -0.002844920533181072
Excitation : [0, 5, 8, 11], Gradient: 0.06775622491894276
Excitation : [0, 5, 9, 10], Gradient: -0.03295294979823385
Excitation : [0, 5, 10, 11], Gradient: 0.002500934809882796
Excitation : [0, 6, 8, 10], Gradient: 0.0
Excitation : [0, 7, 8, 9], Gradient

In [10]:
# End of ADAPT VQE ^ 
# Now use Soran's modified code down below:


symbols = ["O", "H", "H"]
geometry = np.array([[0.028,  0.054,  0.10],
                     [0.986,  1.610, -0.10],
                     [1.855,  0.002,  0.20]], requires_grad=True)

mol = qml.qchem.Molecule(symbols, geometry)
dev = qml.device("default.qubit")

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

In [13]:
def energy(mol):
    @qml.qnode(dev, interface="autograd")
    def circuit(*args):
        
        # note that active_electrons=2, active_orbitals=2 in this example
        qml.BasisState(hf_state, wires=range(qubits))
        
        # apply all single excitations
        for i, singles in enumerate(singles_select):
            qml.SingleExcitation(args[0][0][i], wires=singles)
            
        # apply all double excitations
        for j, doubles in enumerate(doubles_select):
            qml.DoubleExcitation(args[0][0][j + len(singles_select)], wires=doubles)
        
        # note that active_electrons=2, active_orbitals=2 in this example
        H = qml.qchem.molecular_hamiltonian(mol.symbols, mol.coordinates,
            active_electrons=8, active_orbitals=6, args=args[1:])[0]
        
        return qml.expval(H)
    return circuit

In [14]:
# number of zeros should match the number of circuit gates
circuit_param = [np.array([0.0] * (len(singles_select) + len(doubles_select)), requires_grad=True)]

# Starting bond-length and bond-angle:
OH1 = geometry[0] - geometry[1]
OH2 = geometry[0] - geometry[2]
angle = angle_between(OH1, OH2)
OH1_length = np.linalg.norm(OH1) * bohr_angs
print(f"Starting Bond length = {OH1_length:.5f} A, Starting Bond angle = {angle:.5f}" + '\u00b0')
print("")

for n in range(100):
    args = [circuit_param, geometry]
    mol = qml.qchem.Molecule(symbols, geometry)

    g_param = grad(energy(mol), argnum = 0)(*args)
    circuit_param = circuit_param - 0.25 * g_param[0]

    forces = -grad(energy(mol), argnum = 1)(*args)
    geometry = geometry + 0.5 * forces
    
    print(f'n: {n}, E: {energy(mol)(*args):.8f}, Force-max: {abs(forces).max():.8f}')
    
    # Re-Compute molecule characteristics:
    OH1 = geometry[0] - geometry[1]
    OH2 = geometry[0] - geometry[2]
    angle = angle_between(OH1, OH2)
    OH1_length = np.linalg.norm(OH1) * bohr_angs
    print(f"Bond length = {OH1_length:.5f} A, Bond angle = {angle:.5f}" + '\u00b0')
    
    print("")

Starting Bond length = 0.97272 A, Starting Bond angle = 60.64705°

n: 0, E: -74.87663865, Force-max: 0.15378164
Bond length = 0.99182 A, Bond angle = 65.50605°

n: 1, E: -74.92820679, Force-max: 0.12082528
Bond length = 0.99358 A, Bond angle = 70.01465°

n: 2, E: -74.92807378, Force-max: 3.00779971
Bond length = 1.00415 A, Bond angle = 105.14700°



/home/imrannasrullah/anaconda3/envs/venv3.9/lib/python3.9/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: divide by zero encountered in true_divide
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)
/home/imrannasrullah/anaconda3/envs/venv3.9/lib/python3.9/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in multiply
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)
/home/imrannasrullah/anaconda3/envs/venv3.9/lib/python3.9/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in reduce
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)
/home/imrannasrullah/anaconda3/envs/venv3.9/lib/python3.9/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in true_divide
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)


n: 3, E: -74.94164146, Force-max: nan
Bond length = nan A, Bond angle = nan°



LinAlgError: Eigenvalues did not converge